In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import wilcoxon
import numpy as np

# supprime les avertissements de copies de dataframe
# TODO : comprendre et supprimer correctement les avertissements
pd.options.mode.chained_assignment = None

IN = "../../results/iadhore/all/"
OUT = "../../results/python/"
WINDOW_SIZE = 90 # nombre de gènes dans la fenêtre glissante
ANCHORS_MIN = 800
ALPHA = 0.05



In [3]:
"""
Lecture et traitement de la table des multiplicons :
id	genome_x	list_x	parent	genome_y	list_y	level	number_of_anchorpoints	profile_length	begin_x	end_x	begin_y	end_y	is_redundant
"""

df_multiplicons = pd.read_csv(IN + "multiplicons.txt", sep='\t', index_col='id')
print(df_multiplicons[:3], "\nlen:", len(df_multiplicons))


           genome_x list_x  parent         genome_y list_y  level  \
id                                                                  
1   Malus_domestica   Md04     NaN   Prunus_persica   Pp06      2   
2   Malus_domestica   Md12     NaN   Prunus_persica   Pp06      2   
3   Malus_domestica   Md13     NaN  Malus_domestica   Md16      2   

    number_of_anchorpoints  profile_length  begin_x  end_x  begin_y  end_y  \
id                                                                           
1                      831            1304      793   2091     2319   3646   
2                      797            1310      939   2260     2319   3658   
3                      765            1314        0   1337        0   1307   

    is_redundant  
id                
1              0  
2              0  
3             -1   
len: 1202


In [4]:
"""Affichage du tableau du nbr de gènes homologues entre chaque chromosome. """

tmp = pd.concat( [ df_multiplicons['list_x'], df_multiplicons['list_y'] ] )
chromosomes = list( pd.unique(tmp)) 
chromosomes = sorted(chromosomes)
print(chromosomes, "\n")

table_nb_anchors = [[0 for x in range(len(chromosomes))] for x in range(len(chromosomes))]  # table de 0 de taille n*n 
#print(table_nb_anchors)

# remplissage du nombre de points d'ancres entre chaque chromosome
for i in range(len(chromosomes)) :
    for j in range(i, len(chromosomes)) :
        chr1 = chromosomes[i]
        chr2 = chromosomes[j]
        
        rows = df_multiplicons[ ((df_multiplicons.list_x == chr1) & (df_multiplicons.list_y == chr2)) |
            ((df_multiplicons.list_x == chr2) & (df_multiplicons.list_y == chr1)) ]
        anchors = rows['number_of_anchorpoints']
        total = anchors.values.sum()

        table_nb_anchors[i][j] = total
        table_nb_anchors[j][i] = total

#print((table_nb_anchors))

"""Affichage de la heat map du nombre de gènes semblables par paire de chromosomes"""
heat = go.Heatmap(z=table_nb_anchors,
                x=chromosomes,
                y=chromosomes,
                xgap=2, ygap=2
                )
layout = go.Layout(title_text="Heat map of anchor points in chromosomes", title_x=0.5, 
                width=600, height=600,
                xaxis_showgrid=False,
                yaxis_showgrid=False,
                yaxis_autorange='reversed'
                )
   
fig=go.Figure(data=[heat], layout=layout)   
fig.update_xaxes(side="top")

fig.show()




['Md01', 'Md02', 'Md03', 'Md04', 'Md05', 'Md06', 'Md07', 'Md08', 'Md09', 'Md10', 'Md11', 'Md12', 'Md13', 'Md14', 'Md15', 'Md16', 'Md17', 'Pp01', 'Pp02', 'Pp03', 'Pp04', 'Pp05', 'Pp06', 'Pp07', 'Pp08'] 



In [5]:
"""Creation de la df des triplets de chromosomes dont le nbr d'ancres est superieur à un seuil"""
MDchr = chromosomes[:17]
PPchr = chromosomes[17:]
print(MDchr, "\n", PPchr, sep='')

df_triplets = pd.DataFrame(columns=['PP', 'MD1', 'MD2', 'anchorpoints_1', 'anchorpoints_2'])


"""Trouve le 3e chr d'un triplet avec les indices des 2 premiers, en respectant le seuil minimum d'ancres avec PP"""
def third_chr(table_nb_anchors, chromosomes, iPP, iMD1) :
    # MD2 est le chr dont l'indice est celui de la valeur max dans la table des ancres, dans la ligne de MD1, excluant les PP
    max_MD1_MD2 = max(table_nb_anchors[iMD1][:17])
    iMD2 = table_nb_anchors[iMD1].index(max_MD1_MD2)
    MD2 = chromosomes[iMD2]

    if table_nb_anchors[iMD2][iPP] > ANCHORS_MIN : return MD2, iMD2
    else : return None, None

"""Remplissage de la df grâce à la table du nombre d'ancres (gènes dupliqués)"""
def remplissage_df_triplets(table_nb_anchors, chromosomes, df) :
    for i in range(len(chromosomes)) :
        for j in range(i, len(chromosomes)) :

            # on conserve uniquement les matchs entre 2 espèces différentes, et supérieur au seuil 
            if table_nb_anchors[i][j] > ANCHORS_MIN and chromosomes[i][:2] != chromosomes[j][:2] :
                PP = chromosomes[i] if chromosomes[i][:2] == "Pp" else chromosomes[j]
                MD1 = chromosomes[i] if chromosomes[i][:2] == "Md" else chromosomes[j]
                iMD1 = i if chromosomes[i][:2] == "Md" else j # l'indice du chromosome MD1
                iPP = i + j - iMD1 # l'indice du chromosome PP
                
                # on trouve le 3e chromosome du triplet
                MD2, iMD2 = third_chr(table_nb_anchors, chromosomes, iPP, iMD1)
                if MD2 != None : 
                    MD = [MD1, MD2]
                    MD.sort() # trie MD1 et MD2 par ordre alphabetique, pour simplifier la suppression des triplets en doublons après

                    # crée la df d'une ligne et l'ajoute à df_triplets
                    row = pd.DataFrame( {'PP':[PP], 'MD1':[MD[0]], 'MD2':[MD[1]], 'anchorpoints_1':[table_nb_anchors[iPP][iMD1]], 'anchorpoints_2':[table_nb_anchors[iPP][iMD2]]} )
                    df = pd.concat( [df, row], ignore_index=True, axis=0 )

    df.drop_duplicates(subset=['PP', 'MD1', 'MD2'], inplace=True, ignore_index=True) # suppression des triplets en double
    df.sort_values('PP', inplace=True, ignore_index=True) # tri par valeurs de chromosome PP
    return df


df_triplets = remplissage_df_triplets(table_nb_anchors, chromosomes, df_triplets)
print(df_triplets)


['Md01', 'Md02', 'Md03', 'Md04', 'Md05', 'Md06', 'Md07', 'Md08', 'Md09', 'Md10', 'Md11', 'Md12', 'Md13', 'Md14', 'Md15', 'Md16', 'Md17']
['Pp01', 'Pp02', 'Pp03', 'Pp04', 'Pp05', 'Pp06', 'Pp07', 'Pp08']
     PP   MD1   MD2 anchorpoints_1 anchorpoints_2
0  Pp01  Md08  Md15           1291           1506
1  Pp01  Md13  Md16           1568           1538
2  Pp02  Md01  Md07           1143           1619
3  Pp03  Md09  Md17           1494           1468
4  Pp04  Md05  Md10            808            885
5  Pp05  Md06  Md14           1295            900
6  Pp06  Md03  Md11            855            868
7  Pp06  Md04  Md12            928            961
8  Pp07  Md02  Md15            916            998
9  Pp08  Md05  Md10            939            941


In [6]:
"""
Lecture et traitement de la table des multiplicon_pairs :
id	multiplicon		gene_x	gene_y	chr_x chr_y
"""

df_pairs = pd.read_csv(IN + "multiplicon_pairs_modified.txt", sep='\t', index_col='id')
df_pairs.drop(['code'], axis=1, inplace=True) # supprime la colonne 'code'

# jointure pour avoir les noms de chromosome de chaque gene
df_pairs_chr = df_pairs.join(df_multiplicons[['list_x', 'list_y']], on='multiplicon')
df_pairs_chr.rename(columns={'list_x':'chr_x', 'list_y':'chr_y'}, inplace=True)
print(df_pairs_chr[1000:1010])


      multiplicon        gene_x            gene_y chr_x chr_y
id                                                           
1001            2  MD12G1121800  Prupe.6G239600.1  Md12  Pp06
1002            2  MD12G1121900  Prupe.6G240600.1  Md12  Pp06
1003            2  MD12G1122100  Prupe.6G240500.1  Md12  Pp06
1004            2  MD12G1122200  Prupe.6G261400.1  Md12  Pp06
1005            2  MD12G1122400  Prupe.6G240400.1  Md12  Pp06
1006            2  MD12G1122600  Prupe.6G240300.1  Md12  Pp06
1007            2  MD12G1123000  Prupe.6G240200.1  Md12  Pp06
1008            2  MD12G1123100  Prupe.6G240100.1  Md12  Pp06
1009            2  MD12G1123100  Prupe.6G310800.1  Md12  Pp06
1010            2  MD12G1123100  Prupe.6G311300.1  Md12  Pp06


In [7]:
# construction de la liste des triplets de genes chez les chromosomes du triplet
def make_df_genes_triplet(PP, MD1, MD2, df_pairs_chr) :
    
    # récupération les paires de gènes impliquant au moins un gène de PP
    df_PPx = df_pairs_chr[ (df_pairs_chr.chr_x == PP) ] # le gène PP est dans le colonne chr_x
    df_PPy = df_pairs_chr[ (df_pairs_chr.chr_y == PP) ] # le gène PP est dans le colonne chr_y
    # échange de nom 2 colonnes d'une des deux df pour que tous les gènes du chr PP soient dans la meme colonne y
    df_PPx.rename(columns={'gene_x':'gene_y', 'gene_y':'gene_x', 'chr_x':'chr_y', 'chr_y':'chr_x'}, inplace=True)
    # concaténation sur la colonne y (le gène PP) des deux df 
    df_tmp = pd.concat([df_PPy, df_PPx]) 

    # sélection des paires dont le MD correspond à l'un des 2 chr donnés en arguments 
    df_MD1 = df_tmp[ (df_tmp.chr_x == MD1) ]
    df_MD2 = df_tmp[ (df_tmp.chr_x == MD2) ]

    # merge des 2 df, c'est-a-dire réalise une jointure inclusive
    df_triplet = pd.merge(df_MD1, df_MD2, on=['gene_y', 'chr_y'], how='outer', indicator=True)
    df_triplet = df_triplet.filter({'gene_y', 'gene_x_x', 'gene_x_y'}) # conservation des colonnes des gènes PP MD1 et MD2
    df_triplet.rename(columns={'gene_y':'gene_PP', 'gene_x_x':'gene_MD1', 'gene_x_y':'gene_MD2'}, inplace=True) # renomme les colonnes
    df_triplet = df_triplet.reindex(columns=['gene_PP', 'gene_MD1', 'gene_MD2']) # réordonne les colonnes
    df_triplet.sort_values('gene_PP', inplace=True, ignore_index=True) # trie les lignes par PP croissant, ordonné comme sur le chromosome

    #df_triplet.to_csv(OUT + "tmp_" + PP + MD1 + MD2 + ".csv", index=False)

    return df_triplet




In [21]:
"""Création d'une df mesurant le nombre de gènes conservés à chaque itération de la fenêtre le long du chromosome :

iteration    gene_PP          gene_MD1     nb_MD1 rate_MD1    gene_MD2     nb_MD2 rate_MD2
1            Prupe.3G011400.1 MD09G1282900 72.0   80.000000   MD17G1272600 73.0   81.111111  
2            Prupe.3G011500.1 MD09G1282700 73.0   81.111111   MD17G1272500 73.0   81.111111  
3            Prupe.3G011600.1 MD09G1282600 74.0   82.222222   MD17G1272400 73.0   81.111111  
4            Prupe.3G011700.1          NaN 73.0   81.111111   MD17G1272300 73.0   81.111111  
"""

def make_df_fractionation(df_genes) :
    # copie les colonnes des genes MD1 et MD2 dans la new df
    df_fractionation = pd.DataFrame(dtype=float)
    df_fractionation['nb_MD1'] = df_genes['gene_MD1']
    df_fractionation['nb_MD2'] = df_genes['gene_MD2']

    # remplace tous les noms de genes par la valeur 1 et maintient les NaN tel quel
    df_fractionation = df_fractionation.applymap(lambda x: np.nan if pd.isna(x) else 1)

    # compte le nb de genes de MD1 dans la fenetre
    df_fractionation.nb_MD1 = df_fractionation.nb_MD1.rolling(WINDOW_SIZE, min_periods=0).count()
    # compte le nb de genes de MD2 dans la fenetre
    df_fractionation.nb_MD2 = df_fractionation.nb_MD2.rolling(WINDOW_SIZE, min_periods=0).count()
    
    # copies les colonnes de df_genes avant modification 
    df_fractionation = pd.concat([df_fractionation, df_genes[['gene_PP', 'gene_MD1', 'gene_MD2']]], axis=1)

    # calcule les pourcentages de conservation des gènes en fonction de la taille de la fenetre
    df_fractionation['rate_MD1'] = df_fractionation['nb_MD1'] * 100 / WINDOW_SIZE
    df_fractionation['rate_MD2'] = df_fractionation['nb_MD2'] * 100 / WINDOW_SIZE

    # ajoute colonne du numero d'itération de la fenêtre, +1 normal et +1 pour que itération commence à 1 au lieu de 0 
    df_fractionation['iteration'] = df_fractionation.index - WINDOW_SIZE + 2
    # réordonne les colonnes
    df_fractionation = df_fractionation.reindex(columns=['iteration', 'gene_PP', 'gene_MD1', 'nb_MD1', 'rate_MD1', 'gene_MD2', 'nb_MD2', 'rate_MD2']) 
    # remplace par NaN les valeurs de conservation de genes pour les fenêtres du début du chromosome, plus petites que la WINDOW_SIZE
    df_fractionation.loc[:WINDOW_SIZE-2, ['rate_MD1', 'nb_MD1', 'rate_MD2', 'nb_MD2']] = np.nan 
    df_fractionation.loc[:WINDOW_SIZE-2, ['iteration']] = 0 # remplace par 0 les iterations des fenêtres du début

    # converti les valeurs de la colonne iteration en entiers
    df_fractionation = df_fractionation.astype({'iteration': int})    
    
    print(df_fractionation[WINDOW_SIZE-1 :])

    return df_fractionation



In [9]:
"""Affiche le graphique du taux de conservation de genes au sein de 2 chromosomes dupliqués"""
def display_graph_fractionation(df_display, triplet) :
    MD1 = triplet.get("MD1")
    MD2 = triplet.get("MD2")
    PP = triplet.get("PP")

    # renomme les colonnes pour avoir une légende compréhensible
    df = df_display.rename(columns={'rate_MD1': MD1, 'rate_MD2': MD2})
    #print(df)

    # diagramme à ligne brisée du pourcentage de conservation des gènes le long de 2 chromosomes
    fig = px.line(df, x='iteration', y=[MD1, MD2])
    fig.update_layout(xaxis_title="window (size = " + str(WINDOW_SIZE) + ") iteration along " + PP,
                    yaxis_title="genome conservation rate (%)",
                    title="Fractionation biais between " + MD1 + " and " + MD2,
                    xaxis_range=[0, len(df)],
                    yaxis_range=[-1, 101] )
    

    fig.write_html(OUT + MD1 + "_" + MD2 + "_" + PP + ".html")
    fig.show()



In [10]:
"""Parcourt la liste des triplets de chromosomes pour en faire des graphes de biais de fractionnement"""
def analysis_each_triplet(df_triplets) :
    for triplet in df_triplets.to_dict('records') :
       analysis_one_triplet(triplet)

"""Graphe le gene fractionation d'un triplet et réalise son test statistique"""
def analysis_one_triplet(triplet) :
    # traitement des données
    df_genes_triplet = pd.DataFrame()
    df_genes_triplet = make_df_genes_triplet(triplet.get('PP'), triplet.get('MD1'), triplet.get('MD2'), df_pairs_chr)
    df_fractionation = make_df_fractionation(df_genes_triplet)
    # suppression des NaN des données de pourcentage de conservation des gènes
    df_display = df_fractionation.dropna(subset=['rate_MD1', 'rate_MD2'])

    # affichage des données
    print(triplet)
    #print(df_display)
    interpretation_test(df_display)
    display_graph_fractionation(df_display, triplet)

"""Réalise et interprete le test statistique wilcoxons"""
def interpretation_test(df_display) :
    res = wilcoxon(df_display['rate_MD1'], df_display['rate_MD2'])
    print(res)
    if (res.pvalue < ALPHA) : print("TEST SIGNIFICATIF : il existe un biais de fractionnement au risque alpha=", ALPHA, ". ", sep='')
    else : print("TEST NON SIGNIFICATIF : il n'existe pas de biais de fractionnement au risque alpha=", ALPHA, ". ", sep='')


In [22]:
"""test pour le premier triplet de la df contenant PP comme chromosome"""
def test(df_triplets, PP) :
    triplet = (df_triplets[ df_triplets.PP == PP ]).to_dict('records')[0]
    analysis_one_triplet(triplet)


"""MAIN"""
if __name__=="__main__" :
    
    # lance l'analyse de tous les triplets trouvés selon le nbr de gènes similaires
    #analysis_each_triplet(df_triplets)

    # lance l'analyse d'un triplet
    test(df_triplets, "Pp03")

      iteration           gene_PP      gene_MD1  nb_MD1   rate_MD1  \
89            1  Prupe.3G011400.1  MD09G1282900    72.0  80.000000   
90            2  Prupe.3G011500.1  MD09G1282700    73.0  81.111111   
91            3  Prupe.3G011600.1  MD09G1282600    74.0  82.222222   
92            4  Prupe.3G011700.1           NaN    73.0  81.111111   
93            5  Prupe.3G011800.1  MD09G1282200    74.0  82.222222   
...         ...               ...           ...     ...        ...   
2076       1988  Prupe.3G315500.1  MD09G1000800    81.0  90.000000   
2077       1989  Prupe.3G315600.1  MD09G1000600    81.0  90.000000   
2078       1990  Prupe.3G315800.1  MD09G1000400    81.0  90.000000   
2079       1991  Prupe.3G316000.1  MD09G1000200    81.0  90.000000   
2080       1992  Prupe.3G316100.1  MD09G1000300    81.0  90.000000   

          gene_MD2  nb_MD2   rate_MD2  
89    MD17G1272600    73.0  81.111111  
90    MD17G1272500    73.0  81.111111  
91    MD17G1272400    73.0  81.111111  